# Import Libraries and Load Data

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# menampilkan semua kolom
pd.set_option('display.max_columns', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load data
customer=pd.read_csv('/content/drive/MyDrive/Project Data Science Academy/Dataset/customer.csv')
product=pd.read_csv('/content/drive/MyDrive/Project Data Science Academy/Dataset/product.csv', on_bad_lines='skip')
transactions=pd.read_csv('/content/drive/MyDrive/Project Data Science Academy/Dataset/transactions.csv')
click_stream=pd.read_csv('/content/drive/MyDrive/Project Data Science Academy/Dataset/click_stream_new.csv')

# Data Preprocessing

In [ ]:
# melihat missing value dan duplicate value

datasets = [customer, product, transactions, click_stream]
titles = ['customer', 'product', 'transactions', 'click_stream']

data_summary = pd.DataFrame({},)
data_summary['datasets']= titles
data_summary['columns'] = [', '.join([col for col, null in data.isnull().sum().items() ]) for data in datasets]
data_summary['total_rows']= [data.shape[0] for data in datasets]
data_summary['total_cols']= [data.shape[1] for data in datasets]
data_summary['total_duplicate']= [len(data[data.duplicated()]) for data in datasets]
data_summary['total_null']= [data.isnull().sum().sum() for data in datasets]
data_summary['null_cols'] = [', '.join([col for col, null in data.isnull().sum().items() if null > 0]) for data in datasets]
data_summary.style.background_gradient(cmap='Greens_r')

,datasets,columns,total_rows,total_cols,total_duplicate,total_null,null_cols
0,customer,"customer_id, first_name, last_name, username, email, gender, birthdate, device_type, device_id, device_version, home_location_lat, home_location_long, home_location, home_country, first_join_date",100000,15,0,0,
1,product,"id, gender, masterCategory, subCategory, articleType, baseColour, season, year, usage, productDisplayName",44424,10,0,361,"baseColour, season, year, usage, productDisplayName"
2,transactions,"created_at, customer_id, booking_id, session_id, product_metadata, payment_method, payment_status, promo_amount, promo_code, shipment_fee, shipment_date_limit, shipment_location_lat, shipment_location_long, total_amount",852584,14,0,526048,promo_code
3,click_stream,"session_id, event_name, event_time, event_id, traffic_source, event_metadata",895203,6,0,135395,event_metadata


In [ ]:
# melihat lebih spesifik missing value

info=[customer, product, transactions, click_stream]

for i in info:
  print(i.isnull().sum())
  print('*'*40)

customer_id           0
first_name            0
last_name             0
username              0
email                 0
gender                0
birthdate             0
device_type           0
device_id             0
device_version        0
home_location_lat     0
home_location_long    0
home_location         0
home_country          0
first_join_date       0
dtype: int64
****************************************
id                      0
gender                  0
masterCategory          0
subCategory             0
articleType             0
baseColour             15
season                 21
year                    1
usage                 317
productDisplayName      7
dtype: int64
****************************************
created_at                     0
customer_id                    0
booking_id                     0
session_id                     0
product_metadata               0
payment_method                 0
payment_status                 0
promo_amount                   0
promo_co

## Data Cleaning

In [ ]:
product['baseColour'].fillna('No Colour', axis=0, inplace=True)
product['season'].fillna(product['season'].mode()[0], inplace=True)
product['year'].fillna(product['year'].mode()[0], inplace=True)
product['usage'].fillna('Unknown', axis=0, inplace=True)
product['productDisplayName'].fillna('Unknown', axis=0, inplace=True)

In [ ]:
click_stream['event_metadata'].fillna('Unknown', axis=0, inplace=True)

In [ ]:
info=[customer, product, transactions, click_stream]

for i in info:
  print(i.isnull().sum())
  print('*'*40)

customer_id           0
first_name            0
last_name             0
username              0
email                 0
gender                0
birthdate             0
device_type           0
device_id             0
device_version        0
home_location_lat     0
home_location_long    0
home_location         0
home_country          0
first_join_date       0
dtype: int64
****************************************
id                    0
gender                0
masterCategory        0
subCategory           0
articleType           0
baseColour            0
season                0
year                  0
usage                 0
productDisplayName    0
dtype: int64
****************************************
created_at                     0
customer_id                    0
booking_id                     0
session_id                     0
product_metadata               0
payment_method                 0
payment_status                 0
promo_amount                   0
promo_code                52

## Data Integration

In [ ]:
# mengambil data product_id dari kolom product_metadata
transactions['product_id'] = transactions['product_metadata'].str[15:21]
transactions

,created_at,customer_id,booking_id,session_id,product_metadata,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,shipment_location_lat,shipment_location_long,total_amount,product_id
0,2018-07-29T15:22:01.458193Z,5868,186e2bee-0637-4710-8981-50c2d737bc42,3abaa6ce-e320-4e51-9469-d9f3fa328e86,"[{'product_id': 54728, 'quantity': 1, 'item_pr...",Debit Card,Success,1415,WEEKENDSERU,10000,2018-08-03T05:07:24.812676Z,-8.227893,111.969107,199832,54728
1,2018-07-30T12:40:22.365620Z,4774,caadb57b-e808-4f94-9e96-8a7d4c9898db,2ee5ead1-f13e-4759-92df-7ff48475e970,"[{'product_id': 16193, 'quantity': 1, 'item_pr...",Credit Card,Success,0,NaN,10000,2018-08-03T01:29:03.415705Z,3.013470,107.802514,155526,16193
2,2018-09-15T11:51:17.365620Z,4774,6000fffb-9c1a-4f4a-9296-bc8f6b622b50,93325fb6-eb00-4268-bb0e-6471795a0ad0,"[{'product_id': 53686, 'quantity': 4, 'item_pr...",OVO,Success,0,NaN,10000,2018-09-18T08:41:49.422380Z,-2.579428,115.743885,550696,53686
3,2018-11-01T11:23:48.365620Z,4774,f5e530a7-4350-4cd1-a3bc-525b5037bcab,bcad5a61-1b67-448d-8ff4-781d67bc56e4,"[{'product_id': 20228, 'quantity': 1, 'item_pr...",Credit Card,Success,0,NaN,0,2018-11-05T17:42:27.954235Z,-3.602334,120.363824,271012,20228
4,2018-12-18T11:20:30.365620Z,4774,0efc0594-dbbf-4f9a-b0b0-a488cfddf8a2,df1042ab-13e6-4072-b9d2-64a81974c51a,"[{'product_id': 55220, 'quantity': 1, 'item_pr...",Credit Card,Success,0,NaN,0,2018-12-23T17:24:07.361785Z,-3.602334,120.363824,198753,55220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852579,2021-09-21T22:10:05.074873Z,25121,66f02fa4-c62e-497a-8b6e-be239009b50a,484eecde-9f4a-42de-a8df-7ce6668101f4,"[{'product_id': 30642, 'quantity': 2, 'item_pr...",Debit Card,Failed,0,NaN,0,2021-09-27T00:43:40.992167Z,-6.869535,111.077145,226820,30642
852580,2021-12-09T07:35:18.222046Z,34829,ccde6b83-ee42-4cce-b2cd-976574886eac,954e1211-4d45-4d0b-ba84-53e80119592c,"[{'product_id': 19394, 'quantity': 1, 'item_pr...",OVO,Success,5010,AZ2022,10000,2021-12-12T07:52:44.803822Z,-8.677264,115.176947,359203,19394
852581,2021-12-11T01:20:36.944593Z,99645,5bcd3434-1dc6-44bb-ae63-fd19714cbd6c,e6e6c108-abd0-49a2-b0a5-691117f9cfe1,"[{'product_id': 13079, 'quantity': 4, 'item_pr...",Credit Card,Success,2181,WEEKENDSERU,5000,2021-12-12T13:28:22.848703Z,-6.185498,106.831093,1158159,13079
852582,2019-12-18T11:01:09.623382Z,87319,af7f291a-b01b-4bfd-89a9-237b16368b15,bb6e5426-a3f3-426a-b99d-79173eaa234b,"[{'product_id': 11312, 'quantity': 1, 'item_pr...",Credit Card,Success,4503,AZ2022,0,2019-12-23T22:56:46.831720Z,-7.155977,110.891858,184143,11312


In [ ]:
# mengganti nama kolom 'id' menjadi 'product_id' pada dataset product
product = product.rename(columns={'id': 'product_id'})
product

,product_id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt
...,...,...,...,...,...,...,...,...,...,...
44419,17036,Men,Footwear,Shoes,Casual Shoes,White,Summer,2013.0,Casual,Gas Men Caddy Casual Shoe
44420,6461,Men,Footwear,Flip Flops,Flip Flops,Red,Summer,2011.0,Casual,Lotto Men's Soccer Track Flip Flop
44421,18842,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011.0,Casual,Puma Men Graphic Stellar Blue Tshirt
44422,46694,Women,Personal Care,Fragrance,Perfume and Body Mist,Blue,Spring,2017.0,Casual,Rasasi Women Blue Lady Perfume


In [ ]:
# merge customer dan transactions
df = customer.merge(transactions, how='inner', on='customer_id')

# merge df dengan click_stream
df = df.merge(click_stream, how='left', on='session_id')

In [ ]:
# mengubah tipe data product_id pada dataset product
product['product_id'] = product['product_id'].astype(str)

In [ ]:
# merge df dengan product
df = df.merge(product, how='outer', on='product_id')

df

,customer_id,first_name,last_name,username,email,gender_x,birthdate,device_type,device_id,device_version,home_location_lat,home_location_long,home_location,home_country,first_join_date,created_at,booking_id,session_id,product_metadata,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,shipment_location_lat,shipment_location_long,total_amount,product_id,event_name,event_time,event_id,traffic_source,event_metadata,gender_y,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,7279.0,Bakiman,Simanjuntak,3250e5a3-1d23-4675-a647-3281879d42be,3250e5a3_1d23_4675_a647_3281879d42be@startupca...,M,1989-01-23,iOS,d13dde0a-6ae1-43c3-83a7-11bbb922730b,iPad; CPU iPad OS 4_2_1 like Mac OS X,-8.631607,116.428436,Nusa Tenggara Barat,Indonesia,2020-08-23,2020-08-23T12:53:12.916342Z,1fb6f149-05ef-4a22-ac1a-cd5d98cd5510,9d0496a6-37f7-4d0a-8a04-5850a12d00a6,"[{'product_id': 18015, 'quantity': 1, 'item_pr...",LinkAja,Success,2853.0,WEEKENDSERU,15000.0,2020-08-26T05:14:04.908999Z,0.621070,111.101844,844751.0,18015,BOOKING,2020-08-23T12:53:12.916342Z,dbff4928-866d-4be1-b966-e8a447f683dd,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,26815.0,Nyana,Najmudin,e34ca118-75df-4acd-bbd2-0ad2f44c3b3d,e34ca118_75df_4acd_bbd2_0ad2f44c3b3d@zakyfound...,M,1993-06-20,iOS,f2cdddd6-d9bf-4b9d-b767-415bef5efefd,iPhone; CPU iPhone OS 10_3_4 like Mac OS X,-0.106211,114.714124,Kalimantan Tengah,Indonesia,2020-03-24,2020-04-06T02:45:28.079037Z,70d2e956-dd5d-4d40-8b7a-bb07c2657ac4,166616ac-ef2f-41ec-b670-c752a220b8f3,"[{'product_id': 18015, 'quantity': 1, 'item_pr...",OVO,Success,0.0,NaN,10000.0,2020-04-07T05:39:53.855053Z,-6.128435,106.796250,163705.0,18015,BOOKING,2020-04-06T02:45:28.079037Z,8cbaa004-84fe-48c7-a455-97b869de8053,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,69031.0,Caraka,Narpati,bfb7d297-523a-480b-b26a-94ad21bdc7e5,bfb7d297_523a_480b_b26a_94ad21bdc7e5@startupca...,M,2003-02-11,iOS,18e3efa6-aaa3-4a85-ba06-93a08bc7a899,iPhone; CPU iPhone OS 14_2 like Mac OS X,-6.339128,106.825460,Jakarta Raya,Indonesia,2022-04-07,2022-04-07T12:42:02.454218Z,68918b72-a3e9-4225-be84-dc28e232e3df,8dfdd331-3096-4d02-9b63-ed7e736a3e4f,"[{'product_id': 18015, 'quantity': 1, 'item_pr...",OVO,Success,0.0,NaN,50000.0,2022-04-10T13:19:34.537645Z,-1.913940,114.946790,443506.0,18015,BOOKING,2022-04-07T12:42:02.454218Z,9da45f7d-4d62-4754-a129-0a529f14c98d,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,83673.0,Prakosa,Habibi,90037e03-4f79-4655-8d1d-8d21cc424687,90037e03_4f79_4655_8d1d_8d21cc424687@zakyfound...,M,1986-08-15,Android,12332e49-64a7-4684-b1a4-d2164b54fc2e,Android 4.4.3,-6.190635,107.614379,Jawa Barat,Indonesia,2020-10-04,2021-03-02T17:28:59.640623Z,e6a1af23-3f78-4c84-99cf-f3bca1ca0b2c,b386dc5c-961a-4b5b-8768-e539257d6bbb,"[{'product_id': 18015, 'quantity': 1, 'item_pr...",Debit Card,Success,0.0,NaN,15000.0,2021-03-08T03:18:53.378180Z,-6.287592,106.827151,494558.0,18015,BOOKING,2021-03-02T17:28:59.640623Z,4ad695a1-7898-401e-be55-384e52a81182,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,37104.0,Uli,Kuswandari,15221c58-b203-49a1-8644-839fe6f27acb,15221c58_b203_49a1_8644_839fe6f27acb@startupca...,F,1999-08-04,Android,8d74c629-5e7c-444f-937b-59b02b85ad7a,Android 2.2.2,-2.225305,113.837526,Kalimantan Tengah,Indonesia,2018-09-02,2020-04-05T08:55:02.993801Z,40ea9ea4-a3bb-49a4-a6f2-07749ab6f713,a3ad7981-32c3-498a-b40a-1d614f3f1b9b,"[{'product_id': 18015, 'quantity': 1, 'item_pr...",Gopay,Success,0.0,NaN,0.0,2020-04-09T21:01:26.965747Z,-2.225305,113.837526,479178.0,18015,CLICK,2020-04-08T09:52:33.993801Z,aafa4666-338d-4566-827b-3427081a1c1e,MOBILE,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 897008 entries, 0 to 897007
Data columns (total 43 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   customer_id             852584 non-null  float64
 1   first_name              852584 non-null  object 
 2   last_name               852584 non-null  object 
 3   username                852584 non-null  object 
 4   email                   852584 non-null  object 
 5   gender_x                852584 non-null  object 
 6   birthdate               852584 non-null  object 
 7   device_type             852584 non-null  object 
 8   device_id               852584 non-null  object 
 9   device_version          852584 non-null  object 
 10  home_location_lat       852584 non-null  float64
 11  home_location_long      852584 non-null  float64
 12  home_location           852584 non-null  object 
 13  home_country            852584 non-null  object 
 14  first_join_date     

In [ ]:
# mengosongkan timezone UTC -> T
time1 = ['created_at', 'shipment_date_limit', 'event_time']

for i in time1:
  df[i] = df[i].str.replace('T',' ')

In [ ]:
# mengosongkan timezone UTC -> Z
time1 = ['created_at', 'shipment_date_limit', 'event_time']

for i in time1:
  df[i] = df[i].str.replace('Z','')

In [ ]:
# mengubah tipe data menjadi datetime
time = ['birthdate', 'first_join_date', 'created_at', 'shipment_date_limit', 'event_time']

for i in time:
  df[i] = pd.to_datetime(df[i])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 897008 entries, 0 to 897007
Data columns (total 43 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   customer_id             852584 non-null  float64       
 1   first_name              852584 non-null  object        
 2   last_name               852584 non-null  object        
 3   username                852584 non-null  object        
 4   email                   852584 non-null  object        
 5   gender_x                852584 non-null  object        
 6   birthdate               852584 non-null  datetime64[ns]
 7   device_type             852584 non-null  object        
 8   device_id               852584 non-null  object        
 9   device_version          852584 non-null  object        
 10  home_location_lat       852584 non-null  float64       
 11  home_location_long      852584 non-null  float64       
 12  home_location           852584

## Data Transformation

### Feature Engineering

In [ ]:
# feature customer_age
df['customer_age'] = df['event_time'].max() - df['birthdate']
df['customer_age'] = df['customer_age'] / pd.Timedelta(days=365)
df['customer_age'] = round(df['customer_age'], 0)

In [ ]:
# klasifikasi customer_age
def age(x):
  if 0 < x <= 11:
    return 'child'
  elif 11 < x <= 25:
    return 'teenager'
  elif 25 < x <= 55:
    return 'adults'
  else:
    return 'elderly'

df['class_age'] = df['customer_age'].apply(lambda x: age(x))

In [ ]:
df

,customer_id,first_name,last_name,username,email,gender_x,birthdate,device_type,device_id,device_version,home_location_lat,home_location_long,home_location,home_country,first_join_date,created_at,booking_id,session_id,product_metadata,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,shipment_location_lat,shipment_location_long,total_amount,product_id,event_name,event_time,event_id,traffic_source,event_metadata,gender_y,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,customer_age,class_age
0,7279.0,Bakiman,Simanjuntak,3250e5a3-1d23-4675-a647-3281879d42be,3250e5a3_1d23_4675_a647_3281879d42be@startupca...,M,1989-01-23,iOS,d13dde0a-6ae1-43c3-83a7-11bbb922730b,iPad; CPU iPad OS 4_2_1 like Mac OS X,-8.631607,116.428436,Nusa Tenggara Barat,Indonesia,2020-08-23,2020-08-23 12:53:12.916342,1fb6f149-05ef-4a22-ac1a-cd5d98cd5510,9d0496a6-37f7-4d0a-8a04-5850a12d00a6,"[{'product_id': 18015, 'quantity': 1, 'item_pr...",LinkAja,Success,2853.0,WEEKENDSERU,15000.0,2020-08-26 05:14:04.908999,0.621070,111.101844,844751.0,18015,BOOKING,2020-08-23 12:53:12.916342,dbff4928-866d-4be1-b966-e8a447f683dd,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0,adults
1,26815.0,Nyana,Najmudin,e34ca118-75df-4acd-bbd2-0ad2f44c3b3d,e34ca118_75df_4acd_bbd2_0ad2f44c3b3d@zakyfound...,M,1993-06-20,iOS,f2cdddd6-d9bf-4b9d-b767-415bef5efefd,iPhone; CPU iPhone OS 10_3_4 like Mac OS X,-0.106211,114.714124,Kalimantan Tengah,Indonesia,2020-03-24,2020-04-06 02:45:28.079037,70d2e956-dd5d-4d40-8b7a-bb07c2657ac4,166616ac-ef2f-41ec-b670-c752a220b8f3,"[{'product_id': 18015, 'quantity': 1, 'item_pr...",OVO,Success,0.0,NaN,10000.0,2020-04-07 05:39:53.855053,-6.128435,106.796250,163705.0,18015,BOOKING,2020-04-06 02:45:28.079037,8cbaa004-84fe-48c7-a455-97b869de8053,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,adults
2,69031.0,Caraka,Narpati,bfb7d297-523a-480b-b26a-94ad21bdc7e5,bfb7d297_523a_480b_b26a_94ad21bdc7e5@startupca...,M,2003-02-11,iOS,18e3efa6-aaa3-4a85-ba06-93a08bc7a899,iPhone; CPU iPhone OS 14_2 like Mac OS X,-6.339128,106.825460,Jakarta Raya,Indonesia,2022-04-07,2022-04-07 12:42:02.454218,68918b72-a3e9-4225-be84-dc28e232e3df,8dfdd331-3096-4d02-9b63-ed7e736a3e4f,"[{'product_id': 18015, 'quantity': 1, 'item_pr...",OVO,Success,0.0,NaN,50000.0,2022-04-10 13:19:34.537645,-1.913940,114.946790,443506.0,18015,BOOKING,2022-04-07 12:42:02.454218,9da45f7d-4d62-4754-a129-0a529f14c98d,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,teenager
3,83673.0,Prakosa,Habibi,90037e03-4f79-4655-8d1d-8d21cc424687,90037e03_4f79_4655_8d1d_8d21cc424687@zakyfound...,M,1986-08-15,Android,12332e49-64a7-4684-b1a4-d2164b54fc2e,Android 4.4.3,-6.190635,107.614379,Jawa Barat,Indonesia,2020-10-04,2021-03-02 17:28:59.640623,e6a1af23-3f78-4c84-99cf-f3bca1ca0b2c,b386dc5c-961a-4b5b-8768-e539257d6bbb,"[{'product_id': 18015, 'quantity': 1, 'item_pr...",Debit Card,Success,0.0,NaN,15000.0,2021-03-08 03:18:53.378180,-6.287592,106.827151,494558.0,18015,BOOKING,2021-03-02 17:28:59.640623,4ad695a1-7898-401e-be55-384e52a81182,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,adults
4,37104.0,Uli,Kuswandari,15221c58-b203-49a1-8644-839fe6f27acb,15221c58_b203_49a1_8644_839fe6f27acb@startupca...,F,1999-08-04,Android,8d74c629-5e7c-444f-937b-59b02b85ad7a,Android 2.2.2,-2.225305,113.837526,Kalimantan Tengah,Indonesia,2018-09-02,2020-04-05 08:55:02.993801,40ea9ea4-a3bb-49a4-a6f2-07749ab6f713,a3ad7981-32c3-498a-b40a-1d614f3f1b9b,"[{'product_id': 18015, 'quantity': 1, 'item_pr...",Gopay,Success,0.0,NaN,0.0,2020-04-09 21:01:26.965747,-2.225305,113.837526,479178.0,18015,CLICK,2020-04-08 09:52:33.993801,aafa4666-338d-4566-827b-3427081a1c1e,MOBILE,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,teenager
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
# feature total_promo_amount dan total_shipment_fee
a = df.groupby('customer_id')['promo_amount', 'shipment_fee'].sum().reset_index()

a = a.rename(columns={
    'promo_amount': 'total_promo_amount',
    'shipment_fee': 'total_shipment_fee',
    })
a

,customer_id,total_promo_amount,total_shipment_fee
0,3.0,131082.0,365000.0
1,8.0,7699.0,45000.0
2,9.0,0.0,35000.0
3,11.0,9492.0,15000.0
4,15.0,16362.0,50000.0
...,...,...,...
50700,99992.0,12442.0,130000.0
50701,99995.0,63674.0,550000.0
50702,99997.0,0.0,10000.0
50703,99998.0,30325.0,55000.0


In [ ]:
# feature promo_usage_count
b = df.groupby('customer_id')['promo_code'].count().reset_index()

b = b.rename(columns={'promo_code': 'promo_usage_count'})
b

,customer_id,promo_usage_count
0,3.0,27
1,8.0,1
2,9.0,0
3,11.0,1
4,15.0,4
...,...,...
50700,99992.0,2
50701,99995.0,13
50702,99997.0,0
50703,99998.0,6


In [ ]:
# fill yang tidak menggunakan promo code
df['promo_code'].fillna('NONE', axis=0, inplace=True)

In [ ]:
# feature transaction_amount
c1 = df[df['payment_status'] == 'Success']
c = c1.groupby('customer_id')['payment_status'].count().reset_index()

c = c.rename(columns={'payment_status': 'transaction_amount'})
c

,customer_id,transaction_amount
0,3.0,49
1,8.0,7
2,9.0,6
3,11.0,1
4,15.0,5
...,...,...
50237,99992.0,12
50238,99995.0,59
50239,99997.0,1
50240,99998.0,8


In [ ]:
# feature total_money_spent
d = df.groupby('customer_id')['total_amount'].sum().reset_index()

d = d.rename(columns={'total_amount': 'total_money_spent'})
d

,customer_id,total_money_spent
0,3.0,21265889.0
1,8.0,3898561.0
2,9.0,2638665.0
3,11.0,197533.0
4,15.0,2134870.0
...,...,...
50700,99992.0,4352229.0
50701,99995.0,36043743.0
50702,99997.0,226461.0
50703,99998.0,6692540.0


In [ ]:
# feature last_transaction
df = df.sort_values('created_at').drop_duplicates(['customer_id'], keep='last')

df = df.rename(columns={'created_at': 'last_transaction'}).reset_index()

In [ ]:
df

,index,customer_id,first_name,last_name,username,email,gender_x,birthdate,device_type,device_id,device_version,home_location_lat,home_location_long,home_location,home_country,first_join_date,last_transaction,booking_id,session_id,product_metadata,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,shipment_location_lat,shipment_location_long,total_amount,product_id,event_name,event_time,event_id,traffic_source,event_metadata,gender_y,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,customer_age,class_age
0,742610,73773.0,Dacin,Gunawan,b7d339e1-aa83-4bdf-b523-8b850a9455ad,b7d339e1_aa83_4bdf_b523_8b850a9455ad@startupca...,M,1991-02-20,Android,d04602f9-8fc8-405e-ac12-8bc5cce3db69,Android 4.3,-6.578046,108.145161,Jawa Barat,Indonesia,2016-06-30,2016-07-01 22:32:45.218400,5f390336-0a5f-4e0d-bfe1-df080269ce45,8f385c80-1b70-4861-80ac-5eb3b5620fee,"[{'product_id': 32185, 'quantity': 1, 'item_pr...",Debit Card,Success,0.0,NONE,10000.0,2016-07-06 16:02:10.388144,-1.066405,113.575300,256670.0,32185,BOOKING,2016-07-01 22:32:45.218400,5dde66f6-9c9b-4f05-98ac-57483193fac7,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0,adults
1,378988,30771.0,Ghaliyati,Uyainah,eb1ca512-ad85-4c87-9d44-bfbea94e3733,eb1ca512_ad85_4c87_9d44_bfbea94e3733@startupca...,F,2005-05-01,iOS,d7628da3-4b69-4b05-bac1-44993a40cc36,iPad; CPU iPad OS 6_1_6 like Mac OS X,-7.593732,131.616450,Maluku,Indonesia,2016-07-02,2016-07-02 20:01:04.996529,27c08004-141d-4ee8-bf88-63224906e52d,54a40fd2-7446-40b6-97a4-b51600d5df54,"[{'product_id': 18092, 'quantity': 1, 'item_pr...",Credit Card,Success,5272.0,WEEKENDSERU,5000.0,2016-07-06 16:00:51.370901,-7.593732,131.616450,199897.0,18092,BOOKING,2016-07-02 20:01:04.996529,0561963a-5028-4313-b6b3-bc478e0b2bc6,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,teenager
2,760790,67563.0,Melinda,Hasanah,bdc7e701-230b-479a-865d-417a469f4987,bdc7e701_230b_479a_865d_417a469f4987@startupca...,F,2008-12-10,Android,6f8d47ab-e140-492d-af7b-ae1a9b7cbcdb,Android 2.3.4,-6.297146,106.847493,Jakarta Raya,Indonesia,2016-07-03,2016-07-03 16:25:01.401472,34fe80aa-7fe8-4da2-aa19-68a0c8c073b8,2a6aa94f-0b0d-42ce-8615-619cc437dd0c,"[{'product_id': 47432, 'quantity': 3, 'item_pr...",Gopay,Success,0.0,NONE,10000.0,2016-07-09 14:36:17.739418,-6.297146,106.847493,996610.0,47432,BOOKING,2016-07-03 16:25:01.401472,3cd30105-86bd-4931-820b-af60f7d7e5cb,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,teenager
3,203447,22844.0,Queen,Haryanti,dacf34fe-e5bc-4769-8a20-c6f26d72a6d8,dacf34fe_e5bc_4769_8a20_c6f26d72a6d8@zakyfound...,F,1993-04-30,Android,bdd51573-2110-4bc3-ab5d-1789242717f8,Android 5.0,-6.220949,106.783241,Jakarta Raya,Indonesia,2016-07-02,2016-07-04 00:58:13.525069,de1de31a-b71a-4afd-8eec-9946f398b441,8bcd8c49-fe5d-4121-b967-b3964b537c30,"[{'product_id': 25047, 'quantity': 1, 'item_pr...",Gopay,Success,5952.0,AZ2022,10000.0,2016-07-10 01:34:45.509511,-7.435797,108.745902,215548.0,25047,BOOKING,2016-07-04 00:58:13.525069,7bebf1fe-46cb-4524-96e8-1197629792a6,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,adults
4,45007,9914.0,Baktianto,Iswahyudi,cbf3e32b-42ff-4d0c-a3a3-e6e0a1e8aab3,cbf3e32b_42ff_4d0c_a3a3_e6e0a1e8aab3@zakyfound...,M,1996-01-30,iOS,6ee598b2-72f1-457b-b2b5-30efd2e965ba,iPhone; CPU iPhone OS 7_1_2 like Mac OS X,2.274928,99.001182,Sumatera Utara,Indonesia,2016-07-04,2016-07-04 18:10:42.766218,8714130c-06b7-491d-a58a-edf384a77bba,2cb5c61d-a740-472f-92c8-c777e636fc6a,"[{'product_id': 43464, 'quantity': 1, 'item_pr...",Credit Card,Success,6060.0,AZ2022,15000.0,2016-07-06 18:28:35.039732,-2.320134,121.083536,336925.0,43464,BOOKING,2016-07-04 18:10:42.766218,08490bb0-8f6c-4d52-a6e5-1a33eecf9175,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,adults
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
# menghapus kolom index dan customer_id NaN
df = df.drop(['index'], axis=1)

df = df.drop(index=50705)

In [ ]:
df

,customer_id,first_name,last_name,username,email,gender_x,birthdate,device_type,device_id,device_version,home_location_lat,home_location_long,home_location,home_country,first_join_date,last_transaction,booking_id,session_id,product_metadata,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,shipment_location_lat,shipment_location_long,total_amount,product_id,event_name,event_time,event_id,traffic_source,event_metadata,gender_y,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,customer_age,class_age
0,73773.0,Dacin,Gunawan,b7d339e1-aa83-4bdf-b523-8b850a9455ad,b7d339e1_aa83_4bdf_b523_8b850a9455ad@startupca...,M,1991-02-20,Android,d04602f9-8fc8-405e-ac12-8bc5cce3db69,Android 4.3,-6.578046,108.145161,Jawa Barat,Indonesia,2016-06-30,2016-07-01 22:32:45.218400,5f390336-0a5f-4e0d-bfe1-df080269ce45,8f385c80-1b70-4861-80ac-5eb3b5620fee,"[{'product_id': 32185, 'quantity': 1, 'item_pr...",Debit Card,Success,0.0,NONE,10000.0,2016-07-06 16:02:10.388144,-1.066405,113.575300,256670.0,32185,BOOKING,2016-07-01 22:32:45.218400,5dde66f6-9c9b-4f05-98ac-57483193fac7,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0,adults
1,30771.0,Ghaliyati,Uyainah,eb1ca512-ad85-4c87-9d44-bfbea94e3733,eb1ca512_ad85_4c87_9d44_bfbea94e3733@startupca...,F,2005-05-01,iOS,d7628da3-4b69-4b05-bac1-44993a40cc36,iPad; CPU iPad OS 6_1_6 like Mac OS X,-7.593732,131.616450,Maluku,Indonesia,2016-07-02,2016-07-02 20:01:04.996529,27c08004-141d-4ee8-bf88-63224906e52d,54a40fd2-7446-40b6-97a4-b51600d5df54,"[{'product_id': 18092, 'quantity': 1, 'item_pr...",Credit Card,Success,5272.0,WEEKENDSERU,5000.0,2016-07-06 16:00:51.370901,-7.593732,131.616450,199897.0,18092,BOOKING,2016-07-02 20:01:04.996529,0561963a-5028-4313-b6b3-bc478e0b2bc6,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,teenager
2,67563.0,Melinda,Hasanah,bdc7e701-230b-479a-865d-417a469f4987,bdc7e701_230b_479a_865d_417a469f4987@startupca...,F,2008-12-10,Android,6f8d47ab-e140-492d-af7b-ae1a9b7cbcdb,Android 2.3.4,-6.297146,106.847493,Jakarta Raya,Indonesia,2016-07-03,2016-07-03 16:25:01.401472,34fe80aa-7fe8-4da2-aa19-68a0c8c073b8,2a6aa94f-0b0d-42ce-8615-619cc437dd0c,"[{'product_id': 47432, 'quantity': 3, 'item_pr...",Gopay,Success,0.0,NONE,10000.0,2016-07-09 14:36:17.739418,-6.297146,106.847493,996610.0,47432,BOOKING,2016-07-03 16:25:01.401472,3cd30105-86bd-4931-820b-af60f7d7e5cb,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,teenager
3,22844.0,Queen,Haryanti,dacf34fe-e5bc-4769-8a20-c6f26d72a6d8,dacf34fe_e5bc_4769_8a20_c6f26d72a6d8@zakyfound...,F,1993-04-30,Android,bdd51573-2110-4bc3-ab5d-1789242717f8,Android 5.0,-6.220949,106.783241,Jakarta Raya,Indonesia,2016-07-02,2016-07-04 00:58:13.525069,de1de31a-b71a-4afd-8eec-9946f398b441,8bcd8c49-fe5d-4121-b967-b3964b537c30,"[{'product_id': 25047, 'quantity': 1, 'item_pr...",Gopay,Success,5952.0,AZ2022,10000.0,2016-07-10 01:34:45.509511,-7.435797,108.745902,215548.0,25047,BOOKING,2016-07-04 00:58:13.525069,7bebf1fe-46cb-4524-96e8-1197629792a6,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,adults
4,9914.0,Baktianto,Iswahyudi,cbf3e32b-42ff-4d0c-a3a3-e6e0a1e8aab3,cbf3e32b_42ff_4d0c_a3a3_e6e0a1e8aab3@zakyfound...,M,1996-01-30,iOS,6ee598b2-72f1-457b-b2b5-30efd2e965ba,iPhone; CPU iPhone OS 7_1_2 like Mac OS X,2.274928,99.001182,Sumatera Utara,Indonesia,2016-07-04,2016-07-04 18:10:42.766218,8714130c-06b7-491d-a58a-edf384a77bba,2cb5c61d-a740-472f-92c8-c777e636fc6a,"[{'product_id': 43464, 'quantity': 1, 'item_pr...",Credit Card,Success,6060.0,AZ2022,15000.0,2016-07-06 18:28:35.039732,-2.320134,121.083536,336925.0,43464,BOOKING,2016-07-04 18:10:42.766218,08490bb0-8f6c-4d52-a6e5-1a33eecf9175,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,adults
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
# penggabungan seluruh feature
df = df.merge(a, how='left', on='customer_id')
df = df.merge(b, how='left', on='customer_id')
df = df.merge(c, how='left', on='customer_id')
df = df.merge(d, how='left', on='customer_id')

df

,customer_id,first_name,last_name,username,email,gender_x,birthdate,device_type,device_id,device_version,home_location_lat,home_location_long,home_location,home_country,first_join_date,last_transaction,booking_id,session_id,product_metadata,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,shipment_location_lat,shipment_location_long,total_amount,product_id,event_name,event_time,event_id,traffic_source,event_metadata,gender_y,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,customer_age,class_age,total_promo_amount,total_shipment_fee,promo_usage_count,transaction_amount,total_money_spent
0,73773.0,Dacin,Gunawan,b7d339e1-aa83-4bdf-b523-8b850a9455ad,b7d339e1_aa83_4bdf_b523_8b850a9455ad@startupca...,M,1991-02-20,Android,d04602f9-8fc8-405e-ac12-8bc5cce3db69,Android 4.3,-6.578046,108.145161,Jawa Barat,Indonesia,2016-06-30,2016-07-01 22:32:45.218400,5f390336-0a5f-4e0d-bfe1-df080269ce45,8f385c80-1b70-4861-80ac-5eb3b5620fee,"[{'product_id': 32185, 'quantity': 1, 'item_pr...",Debit Card,Success,0.0,NONE,10000.0,2016-07-06 16:02:10.388144,-1.066405,113.575300,256670.0,32185,BOOKING,2016-07-01 22:32:45.218400,5dde66f6-9c9b-4f05-98ac-57483193fac7,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0,adults,0.0,10000.0,0,1.0,256670.0
1,30771.0,Ghaliyati,Uyainah,eb1ca512-ad85-4c87-9d44-bfbea94e3733,eb1ca512_ad85_4c87_9d44_bfbea94e3733@startupca...,F,2005-05-01,iOS,d7628da3-4b69-4b05-bac1-44993a40cc36,iPad; CPU iPad OS 6_1_6 like Mac OS X,-7.593732,131.616450,Maluku,Indonesia,2016-07-02,2016-07-02 20:01:04.996529,27c08004-141d-4ee8-bf88-63224906e52d,54a40fd2-7446-40b6-97a4-b51600d5df54,"[{'product_id': 18092, 'quantity': 1, 'item_pr...",Credit Card,Success,5272.0,WEEKENDSERU,5000.0,2016-07-06 16:00:51.370901,-7.593732,131.616450,199897.0,18092,BOOKING,2016-07-02 20:01:04.996529,0561963a-5028-4313-b6b3-bc478e0b2bc6,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,teenager,5272.0,5000.0,1,1.0,199897.0
2,67563.0,Melinda,Hasanah,bdc7e701-230b-479a-865d-417a469f4987,bdc7e701_230b_479a_865d_417a469f4987@startupca...,F,2008-12-10,Android,6f8d47ab-e140-492d-af7b-ae1a9b7cbcdb,Android 2.3.4,-6.297146,106.847493,Jakarta Raya,Indonesia,2016-07-03,2016-07-03 16:25:01.401472,34fe80aa-7fe8-4da2-aa19-68a0c8c073b8,2a6aa94f-0b0d-42ce-8615-619cc437dd0c,"[{'product_id': 47432, 'quantity': 3, 'item_pr...",Gopay,Success,0.0,NONE,10000.0,2016-07-09 14:36:17.739418,-6.297146,106.847493,996610.0,47432,BOOKING,2016-07-03 16:25:01.401472,3cd30105-86bd-4931-820b-af60f7d7e5cb,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,teenager,0.0,10000.0,0,1.0,996610.0
3,22844.0,Queen,Haryanti,dacf34fe-e5bc-4769-8a20-c6f26d72a6d8,dacf34fe_e5bc_4769_8a20_c6f26d72a6d8@zakyfound...,F,1993-04-30,Android,bdd51573-2110-4bc3-ab5d-1789242717f8,Android 5.0,-6.220949,106.783241,Jakarta Raya,Indonesia,2016-07-02,2016-07-04 00:58:13.525069,de1de31a-b71a-4afd-8eec-9946f398b441,8bcd8c49-fe5d-4121-b967-b3964b537c30,"[{'product_id': 25047, 'quantity': 1, 'item_pr...",Gopay,Success,5952.0,AZ2022,10000.0,2016-07-10 01:34:45.509511,-7.435797,108.745902,215548.0,25047,BOOKING,2016-07-04 00:58:13.525069,7bebf1fe-46cb-4524-96e8-1197629792a6,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,adults,5952.0,10000.0,1,1.0,215548.0
4,9914.0,Baktianto,Iswahyudi,cbf3e32b-42ff-4d0c-a3a3-e6e0a1e8aab3,cbf3e32b_42ff_4d0c_a3a3_e6e0a1e8aab3@zakyfound...,M,1996-01-30,iOS,6ee598b2-72f1-457b-b2b5-30efd2e965ba,iPhone; CPU iPhone OS 7_1_2 like Mac OS X,2.274928,99.001182,Sumatera Utara,Indonesia,2016-07-04,2016-07-04 18:10:42.766218,8714130c-06b7-491d-a58a-edf384a77bba,2cb5c61d-a740-472f-92c8-c777e636fc6a,"[{'product_id': 43464, 'quantity': 1, 'item_pr...",Credit Card,Success,6060.0,AZ2022,15000.0,2016-07-06 18:28:35.039732,-2.320134,121.083536,336925.0,43464,BOOKING,2016-07-04 18:10:42.766218,08490bb0-8f6c-4d52-a6e5-1a33eecf9175,MOBILE,{

### Variable is_loyal

In [ ]:
today = df['last_transaction'].max()
today

Timestamp('2022-07-31 23:59:45.821469')

In [ ]:
df['range_days'] = df['last_transaction'].apply(lambda x: (today - x).days)

In [ ]:
# pembentukan variabel is_loyal
e = df['range_days'] > 61
f = df['total_money_spent'] < df['total_money_spent'].mean()
df['is_loyal'] = np.where((e) & (f), 0, 1)

In [ ]:
df['is_loyal'].value_counts()

0    29757
1    20948
Name: is_loyal, dtype: int64

In [ ]:
df

,customer_id,first_name,last_name,username,email,gender_x,birthdate,device_type,device_id,device_version,home_location_lat,home_location_long,home_location,home_country,first_join_date,last_transaction,booking_id,session_id,product_metadata,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,shipment_location_lat,shipment_location_long,total_amount,product_id,event_name,event_time,event_id,traffic_source,event_metadata,gender_y,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,customer_age,class_age,total_promo_amount,total_shipment_fee,promo_usage_count,transaction_amount,total_money_spent,range_days,is_loyal
0,73773.0,Dacin,Gunawan,b7d339e1-aa83-4bdf-b523-8b850a9455ad,b7d339e1_aa83_4bdf_b523_8b850a9455ad@startupca...,M,1991-02-20,Android,d04602f9-8fc8-405e-ac12-8bc5cce3db69,Android 4.3,-6.578046,108.145161,Jawa Barat,Indonesia,2016-06-30,2016-07-01 22:32:45.218400,5f390336-0a5f-4e0d-bfe1-df080269ce45,8f385c80-1b70-4861-80ac-5eb3b5620fee,"[{'product_id': 32185, 'quantity': 1, 'item_pr...",Debit Card,Success,0.0,NONE,10000.0,2016-07-06 16:02:10.388144,-1.066405,113.575300,256670.0,32185,BOOKING,2016-07-01 22:32:45.218400,5dde66f6-9c9b-4f05-98ac-57483193fac7,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0,adults,0.0,10000.0,0,1.0,256670.0,2221,0
1,30771.0,Ghaliyati,Uyainah,eb1ca512-ad85-4c87-9d44-bfbea94e3733,eb1ca512_ad85_4c87_9d44_bfbea94e3733@startupca...,F,2005-05-01,iOS,d7628da3-4b69-4b05-bac1-44993a40cc36,iPad; CPU iPad OS 6_1_6 like Mac OS X,-7.593732,131.616450,Maluku,Indonesia,2016-07-02,2016-07-02 20:01:04.996529,27c08004-141d-4ee8-bf88-63224906e52d,54a40fd2-7446-40b6-97a4-b51600d5df54,"[{'product_id': 18092, 'quantity': 1, 'item_pr...",Credit Card,Success,5272.0,WEEKENDSERU,5000.0,2016-07-06 16:00:51.370901,-7.593732,131.616450,199897.0,18092,BOOKING,2016-07-02 20:01:04.996529,0561963a-5028-4313-b6b3-bc478e0b2bc6,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,teenager,5272.0,5000.0,1,1.0,199897.0,2220,0
2,67563.0,Melinda,Hasanah,bdc7e701-230b-479a-865d-417a469f4987,bdc7e701_230b_479a_865d_417a469f4987@startupca...,F,2008-12-10,Android,6f8d47ab-e140-492d-af7b-ae1a9b7cbcdb,Android 2.3.4,-6.297146,106.847493,Jakarta Raya,Indonesia,2016-07-03,2016-07-03 16:25:01.401472,34fe80aa-7fe8-4da2-aa19-68a0c8c073b8,2a6aa94f-0b0d-42ce-8615-619cc437dd0c,"[{'product_id': 47432, 'quantity': 3, 'item_pr...",Gopay,Success,0.0,NONE,10000.0,2016-07-09 14:36:17.739418,-6.297146,106.847493,996610.0,47432,BOOKING,2016-07-03 16:25:01.401472,3cd30105-86bd-4931-820b-af60f7d7e5cb,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,teenager,0.0,10000.0,0,1.0,996610.0,2219,0
3,22844.0,Queen,Haryanti,dacf34fe-e5bc-4769-8a20-c6f26d72a6d8,dacf34fe_e5bc_4769_8a20_c6f26d72a6d8@zakyfound...,F,1993-04-30,Android,bdd51573-2110-4bc3-ab5d-1789242717f8,Android 5.0,-6.220949,106.783241,Jakarta Raya,Indonesia,2016-07-02,2016-07-04 00:58:13.525069,de1de31a-b71a-4afd-8eec-9946f398b441,8bcd8c49-fe5d-4121-b967-b3964b537c30,"[{'product_id': 25047, 'quantity': 1, 'item_pr...",Gopay,Success,5952.0,AZ2022,10000.0,2016-07-10 01:34:45.509511,-7.435797,108.745902,215548.0,25047,BOOKING,2016-07-04 00:58:13.525069,7bebf1fe-46cb-4524-96e8-1197629792a6,MOBILE,{'payment_status': 'Success'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,adults,5952.0,10000.0,1,1.0,215548.0,2218,0
4,9914.0,Baktianto,Iswahyudi,cbf3e32b-42ff-4d0c-a3a3-e6e0a1e8aab3,cbf3e32b_42ff_4d0c_a3a3_e6e0a1e8aab3@zakyfound...,M,1996-01-30,iOS,6ee598b2-72f1-457b-b2b5-30efd2e965ba,iPhone; CPU iPhone OS 7_1_2 like Mac OS X,2.274928,99.001182,Sumatera Utara,Indonesia,2016-07-04,2016-07-04 18:10:42.766218,8714130c-06b7-491d-a58a-edf384a77bba,2cb5c61d-a740-472f-92c8-c777e636fc6a,"[{'product_id': 43464, 'quantity': 1, 'item_pr...",Credit Card,Success,6060.0,AZ2022,15000.0,2016-07-06 18:28:35.039732,-2.320134,121.083536,336925.0,43464,BOOKING,2016-07-04 18:10:42.7662

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50705 entries, 0 to 50704
Data columns (total 52 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   customer_id             50705 non-null  float64       
 1   first_name              50705 non-null  object        
 2   last_name               50705 non-null  object        
 3   username                50705 non-null  object        
 4   email                   50705 non-null  object        
 5   gender_x                50705 non-null  object        
 6   birthdate               50705 non-null  datetime64[ns]
 7   device_type             50705 non-null  object        
 8   device_id               50705 non-null  object        
 9   device_version          50705 non-null  object        
 10  home_location_lat       50705 non-null  float64       
 11  home_location_long      50705 non-null  float64       
 12  home_location           50705 non-null  object

# Export Data

In [ ]:
df.to_csv('data_1.csv', index=False)

In [ ]:
df['payment_method'].value_counts()

Credit Card    17868
Gopay          10164
OVO            10048
Debit Card      8179
LinkAja         4446
Name: payment_method, dtype: int64